# Задача

Делаем анализ, который обсуждали.

*Исходник данных*: продажи за 2 года.

1. На 2016.12.31 получаем инфу по кол-ву заказов на каждого юзера. Подсчитываем сколько пользователей попало в группу с 1 заказом, 2, 3 и т.д.
2. За январь 2017 смотрим, какое кол-во юзеров из п.1 купило в каждой группе и на какую сумму за месяц.(скрипты вам должны помочь)
3. Подсчитываем вероятность перехода в покупающего по каждой группе и ср траты за январь в группе (как на занятии).
4. В идеале сделать тоже самое на февраль, март, и до конца 17 года (чтобы оценить сезонность и рост трат за месяц)

# Подготовка

## Подключение библиотек

In [11]:
import pymysql.cursors
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
import getpass

## Подключение к базе данных

In [44]:
# password_mysql = getpass.getpass(prompt="Введите пароль от MySQL-сервера: ")
password_mysql = "pass123"
mydb = pymysql.connect(host="127.0.0.1",
                             user='root',
                             password=password_mysql, 
                             db='db_analyt_loc',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
 
print ("connect successful!!")

connect successful!!


In [15]:
def show_tables():
    return pd.read_sql("""SHOW TABLES FROM db_analyt_loc;""", con=mydb)
show_tables()

,Tables_in_db_analyt_loc
0,orders_20190822
1,orders_all
2,time_user_b
3,time_user_ym_s
4,users_for_merg


In [17]:
mydb.cursor().execute(
    """
    DROP TABLE IF EXISTS one_month_table;
    """)
mydb.commit()

In [18]:
%%time
mydb.cursor().execute(
    """
    CREATE TABLE one_month_table (
    SELECT 
        user_id,
        COUNT(id_o) AS n,
        MONTH(o_date) AS date_m,
        YEAR(o_date) AS date_y
    FROM orders_20190822 o
    WHERE price BETWEEN 0 AND 150000
    GROUP BY user_id, date_m, date_y);
    """)
mydb.commit()

Wall time: 23.9 s


In [20]:
%%time
df_one_month_table = pd.read_sql(f"SELECT * FROM one_month_table;", con=mydb)
df_one_month_table.head()

Wall time: 37.2 s


,user_id,n,date_m,date_y
0,337544,1,1,2016
1,171642,4,1,2016
2,260596,1,1,2016
3,1105609,1,1,2016
4,982696,1,1,2016


### 
-- Группировка пользователей на грууппы в зависимости от количества заказов
-- Разделение по группам кол-во покупак в разные месяцы до 20170101.

In [21]:
%%time
df_before_20170101 = pd.read_sql(
    """
    SELECT 
        COUNT(omt.user_id) AS count_users,
        CASE 
            WHEN n < 5 THEN 1
            WHEN n BETWEEN 5 AND 10 THEN 2
            WHEN n BETWEEN 11 AND 20 THEN 3
            WHEN n > 20 THEN 4 END AS buyer_group,
        date_m,
        date_y,
        sum(o.price) AS summ
    FROM one_month_table omt
    LEFT JOIN orders_20190822 AS o
    ON omt.user_id = o.user_id
    WHERE 
        o.price BETWEEN 0 AND 150000 AND 
        o.o_date < DATE('2017-01-01') AND
        DATE(concat(omt.date_y,'-',omt.date_m,'-','01')) < DATE('2017-01-01')
    GROUP BY buyer_group, omt.date_m, omt.date_y;
    """, con=mydb)
df_before_20170101.head()

Wall time: 11.6 s


,count_users,buyer_group,date_m,date_y,summ
0,133283,1,1,2016,2.503092e+08
1,9309,2,1,2016,1.337300e+07
2,1196,3,1,2016,1.292315e+06
3,22401,4,1,2016,6.637888e+07
4,134305,1,2,2016,2.503801e+08


### -- просмотр, в какой групе был пользователь, и в какую группу он попал

In [22]:
%%time
df_groups = pd.read_sql(
    """
    SELECT 
        ot.user_id, 
        expenses,
        buyer_group_prev,
        CASE 
            WHEN buyer_group_cur < 5 THEN 1
            WHEN buyer_group_cur BETWEEN 5 AND 10 THEN 2
            WHEN buyer_group_cur BETWEEN 11 AND 20 THEN 3
            WHEN buyer_group_cur > 20 THEN 4 END AS buyer_group_cur
    FROM (
        SELECT 
            DISTINCT(user_id),
            CASE 
                WHEN n < 5 THEN 1
                WHEN n BETWEEN 5 AND 10 THEN 2
                WHEN n BETWEEN 11 AND 20 THEN 3
                WHEN n > 20 THEN 4 END AS buyer_group_prev
        FROM one_month_table AS ot
        WHERE DATE(concat(date_y,'-',date_m,'-','01')) < DATE('2017-01-01')) ot
    INNER JOIN (
        SELECT
            user_id,
            sum(price) AS expenses,
            count(id_o) AS buyer_group_cur
        FROM orders_20190822 oh
        WHERE YEAR(o_date) = 2017 AND
              MONTH(o_date) = 1 AND
              price BETWEEN 0 AND 150000
        GROUP BY user_id) oh ON ot.user_id = oh.user_id;
    """, con=mydb)
df_groups.head()

Wall time: 6.61 s


,user_id,expenses,buyer_group_prev,buyer_group_cur
0,655099,208.6,1,1
1,278321,1897.0,1,1
2,340809,6509.3,1,2
3,340809,6509.3,2,2
4,2654584,989.8,1,1


### -- просмотр, сколько пользователей из группп в прошлом сохранились в анализируемом месяце

In [45]:
%%time
df_dynamic = pd.read_sql(
    """
    SELECT
        buyer_group_prev,
        count(ot.user_id) AS осталось,
        count(ot.user_id) / (
                SELECT count(user_id) 
                FROM one_month_table ot2 
                WHERE n = ot.buyer_group_prev AND
                        DATE(concat(date_y,'-',date_m,'-','01')) < DATE('2016-12-01'))*100 '%'
    FROM (
        SELECT *,
            CASE 
                WHEN n < 5 THEN 1
                WHEN n BETWEEN 5 AND 10 THEN 2
                WHEN n BETWEEN 11 AND 20 THEN 3
                WHEN n > 20 THEN 4 END AS buyer_group_prev
        FROM one_month_table AS ot
        WHERE DATE(concat(date_y,'-',date_m,'-','01')) < DATE('2016-12-01')) ot
    INNER JOIN (
        SELECT
            user_id,
            sum(price) AS expenses,
            count(id_o) AS buyer_group_cur
        FROM orders_20190822 oh
        WHERE YEAR(o_date) = 2017 AND
              MONTH(o_date) = 1 AND
              price BETWEEN 0 AND 150000
        GROUP BY user_id) oh 
    ON ot.user_id = oh.user_id
    GROUP BY buyer_group_prev; 
    """, con=mydb)
df_dynamic.head()

Wall time: 13.9 s


,buyer_group_prev,осталось,%
0,1,60412,14.1795
1,2,3485,5.0334
2,3,436,2.2014
3,4,280,3.7318


In [46]:
df_dynamic['period'] = 1701
df_dynamic.head()

,buyer_group_prev,осталось,%,period
0,1,60412,14.1795,1701
1,2,3485,5.0334,1701
2,3,436,2.2014,1701
3,4,280,3.7318,1701


# Закрытие сессии

In [47]:
mydb.close()